In [1]:
import pandas as pd

## Читає файл та змінює назви стовпців

In [2]:
df = pd.read_html('data/data.html')[0]
df.columns = ['Temp', 'L_value', 'R_value', 'A_M_value', 'Color_value', 'Spectral_Class_value', 'Type_value']

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temp                  261 non-null    int64  
 1   L_value               259 non-null    float64
 2   R_value               258 non-null    float64
 3   A_M_value             258 non-null    float64
 4   Color_value           260 non-null    object 
 5   Spectral_Class_value  259 non-null    object 
 6   Type_value            258 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 14.4+ KB


df.describe()

## Перевірка на наявність пропущених значень

In [4]:
print(df.isnull().sum())

Temp                    0
L_value                 2
R_value                 3
A_M_value               3
Color_value             1
Spectral_Class_value    2
Type_value              3
dtype: int64


## Заповнення пропущених значень

In [5]:
# Заповнення пропущених значень в числових стовпцях середнім значенням цих стовпців
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    df[col].fillna(df[col].mean(), inplace=True)

# Заповнення пропущених значень в нечислових стовпцях модою (найчастіше зустрічається значення) цих стовпців
for col in df.select_dtypes(include=['object']).columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

## Перевірка аномальних значень

In [6]:
df_original = df.copy()
numeric_cols = df.select_dtypes(include=['float64', 'int64'])

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    # Визначення меж викидів
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Виявлення викидів
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"BEFORE: Outliers in {col}: {len(outliers)}")

    # Заміна значень викидів на медіану
    df.loc[df[col] < lower_bound, col] = df[col].median()
    df.loc[df[col] > upper_bound, col] = df[col].median()

    # Кількість викидів після заміни
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"AFTER: Outliers in {col}: {len(outliers)}\n")

# До / після
diff = df_original.compare(df)
print(f'\n{diff}')

BEFORE: Outliers in Temp: 9
AFTER: Outliers in Temp: 0

BEFORE: Outliers in L_value: 13
AFTER: Outliers in L_value: 0

BEFORE: Outliers in R_value: 47
AFTER: Outliers in R_value: 0

BEFORE: Outliers in A_M_value: 0
AFTER: Outliers in A_M_value: 0

BEFORE: Outliers in Type_value: 0
AFTER: Outliers in Type_value: 0


        Temp           L_value             R_value      
        self   other      self other          self other
30   39000.0  6850.0       NaN   NaN           NaN   NaN
50       NaN     NaN       NaN   NaN   1520.000000   1.8
51       NaN     NaN       NaN   NaN   1260.000000   1.8
52       NaN     NaN       NaN   NaN   1183.000000   1.8
53       NaN     NaN  550000.0  1.43   1648.000000   1.8
54       NaN     NaN       NaN   NaN   1324.000000   1.8
55       NaN     NaN       NaN   NaN   1349.000000   1.8
56       NaN     NaN       NaN   NaN   1673.000000   1.8
57       NaN     NaN       NaN   NaN   1284.000000   1.8
58       NaN     NaN       NaN   NaN    955.000000   1.8

## Перевірка на наявність дублікатів та їх видалення

In [7]:
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)

19
